In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def load_and_create_dataset(data_folders):
    X, y = [], []
    for label, data_folder in enumerate(data_folders):
        for filename in os.listdir(data_folder):
            img_path = os.path.join(data_folder, filename)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))  # Resize images to (224, 224)
                X.append(img)
                y.append(label)
            except Exception as e:
                print(f"Error loading image: {img_path}, Error: {str(e)}")
    X = np.array(X) / 255.0  # Normalize images
    y = np.array(y)

    # Create adjacency matrix
    adj_matrix = create_adjacency_matrix(X[0].shape)

    return X, y, adj_matrix


In [3]:
def create_adjacency_matrix(img_shape):
    rows, cols, _ = img_shape
    num_nodes = rows * cols
    adj_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        row = i // cols
        col = i % cols
        neighbors = []
        if row > 0:
            neighbors.append((row - 1, col))  # Top
        if row < rows - 1:
            neighbors.append((row + 1, col))  # Bottom
        if col > 0:
            neighbors.append((row, col - 1))  # Left
        if col < cols - 1:
            neighbors.append((row, col + 1))  # Right
        for n_row, n_col in neighbors:
            j = n_row * cols + n_col
            adj_matrix[i][j] = 1
            adj_matrix[j][i] = 1
    return adj_matrix

In [4]:
def GCN_model(input_shape, adj_matrix):
    inputs = layers.Input(shape=input_shape)

    # Reshape input to match the adjacency matrix shape
    x = layers.Reshape((-1, input_shape[0] * input_shape[1]))(inputs)

    # Graph convolutional layer
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

    # Graph pooling layer
    pooled = layers.GlobalAveragePooling1D()(x)

    # Fully connected layer
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(pooled)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
# Paths to augmented folders
augmented_class_0_path = r"E:\dataset_temp\augmented_class_0"
augmented_class_1_path = r"E:\dataset_temp\augmented_class_1"

# Load and create the dataset
X, y, adj_matrix = load_and_create_dataset([augmented_class_0_path, augmented_class_1_path])

# Split the dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16)

# Print counts
print("Train set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

MemoryError: Unable to allocate 18.8 GiB for an array with shape (50176, 50176) and data type float64

In [ ]:
# Define input shape
input_shape = X_train[0].shape

# Create and compile the model
model = GCN_model(input_shape, adj_matrix)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[early_stopping])


In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Save the mode
model.save("GCN_model_copy1.h5")


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def predict_on_images(model, image_folder):
    predictions = []
    class_labels = ['not risky', 'risky']
    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize images to the same size as the training images
            img = np.expand_dims(img, axis=0)  # Add batch dimension
            pred = model.predict(img)
            predicted_class = np.round(pred).astype(int)[0][0]  # Round the prediction to 0 or 1
            predictions.append((filename, predicted_class))
            # Display the image with its predicted class
            plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
            plt.title(f"Predicted class: {class_labels[predicted_class]}")
            plt.axis('off')
            plt.show()
    return predictions

# Call the function with your loaded model and test image folder
model_load_path = "GCN_model_copy1.h5"
model = tf.keras.models.load_model(model_load_path)
image_folder = r"E:\dataset_temp\test"
predictions = predict_on_images(model, image_folder)
print(predictions)